In [1]:
import pandas as pd
import numpy as np
from scipy.interpolate import CubicSpline, UnivariateSpline, InterpolatedUnivariateSpline, interp1d
from datetime import datetime
from math import floor
import matplotlib.pyplot as plt
from BEE_WEATHER_DATA import BROODMINDER_GET, AMBIENT_GET, READ_HIVE, PROCESS_HIVE, READ_BEE_WEATHER, PROCESS_BEE_WEATHER, PROCESS_AMBIENT, GRAPH_DATA, GET_MOON_IMAGE, GET_FORECAST

hive_name = "New Left Hive"
key = 0

if key == 1:
    Data = PROCESS_AMBIENT()
    #display(Data)
    GRAPH_DATA(Data)
if key == 2:
    Data = PROCESS_BEE_WEATHER()
    #display(Data)
    GRAPH_DATA(Data)
if key == 3:
    Data = PROCESS_HIVE(hive_name, interp=1)
    #display(Data)
    [GRAPH_DATA(Data[str(i)]) for i in Data.keys()]

#pix = GET_MOON_IMAGE(216, save=1)
#plt.imshow(pix)
#plt.show()

data = GET_FORECAST()
print(data)

print("Done")

{'cod': '200', 'message': 0, 'cnt': 40, 'list': [{'dt': 1710255600, 'main': {'temp': 44.06, 'feels_like': 44.06, 'temp_min': 44.02, 'temp_max': 44.06, 'pressure': 1014, 'sea_level': 1014, 'grnd_level': 868, 'humidity': 60, 'temp_kf': 0.02}, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10d'}], 'clouds': {'all': 100}, 'wind': {'speed': 2.15, 'deg': 337, 'gust': 2.08}, 'visibility': 10000, 'pop': 0.36, 'rain': {'3h': 0.13}, 'sys': {'pod': 'd'}, 'dt_txt': '2024-03-12 15:00:00'}, {'dt': 1710266400, 'main': {'temp': 43.12, 'feels_like': 40.95, 'temp_min': 41.27, 'temp_max': 43.12, 'pressure': 1014, 'sea_level': 1014, 'grnd_level': 868, 'humidity': 67, 'temp_kf': 1.03}, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10d'}], 'clouds': {'all': 100}, 'wind': {'speed': 3.98, 'deg': 241, 'gust': 6.98}, 'visibility': 7185, 'pop': 0.56, 'rain': {'3h': 0.56}, 'sys': {'pod': 'd'}, 'dt_txt': '2024-03-12 18:00:00'}, {'dt': 1710277200, 'main

In [73]:
response = {}
def recursive_dict(data):
    for i, key in enumerate(data):
        if type(data[key]) == type({}):
            response.update({key: pd.DataFrame(data[key])})
        else:
            response.update({key: data[key]})
            
display(response)

{'cod': '200',
 'message': 0,
 'cnt': 40,
 'list': [{'dt': 1710255600,
   'main': {'temp': 44.06,
    'feels_like': 44.06,
    'temp_min': 44.02,
    'temp_max': 44.06,
    'pressure': 1014,
    'sea_level': 1014,
    'grnd_level': 868,
    'humidity': 60,
    'temp_kf': 0.02},
   'weather': [{'id': 500,
     'main': 'Rain',
     'description': 'light rain',
     'icon': '10d'}],
   'clouds': {'all': 100},
   'wind': {'speed': 2.15, 'deg': 337, 'gust': 2.08},
   'visibility': 10000,
   'pop': 0.36,
   'rain': {'3h': 0.13},
   'sys': {'pod': 'd'},
   'dt_txt': '2024-03-12 15:00:00'},
  {'dt': 1710266400,
   'main': {'temp': 43.12,
    'feels_like': 40.95,
    'temp_min': 41.27,
    'temp_max': 43.12,
    'pressure': 1014,
    'sea_level': 1014,
    'grnd_level': 868,
    'humidity': 67,
    'temp_kf': 1.03},
   'weather': [{'id': 500,
     'main': 'Rain',
     'description': 'light rain',
     'icon': '10d'}],
   'clouds': {'all': 100},
   'wind': {'speed': 3.98, 'deg': 241, 'gust': 6.9